In [51]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
from  plotly.subplots  import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt

df = pd.read_csv('superstore.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Category        51290 non-null  object 
 1   City            51290 non-null  object 
 2   Country         51290 non-null  object 
 3   Customer.ID     51290 non-null  object 
 4   Customer.Name   51290 non-null  object 
 5   Discount        51290 non-null  float64
 6   Market          51290 non-null  object 
 7   记录数             51290 non-null  int64  
 8   Order.Date      51290 non-null  object 
 9   Order.ID        51290 non-null  object 
 10  Order.Priority  51290 non-null  object 
 11  Product.ID      51290 non-null  object 
 12  Product.Name    51290 non-null  object 
 13  Profit          51290 non-null  float64
 14  Quantity        51290 non-null  int64  
 15  Region          51290 non-null  object 
 16  Row.ID          51290 non-null  int64  
 17  Sales           51290 non-null 

### APAC 시장 점유율 높은 5개 국가

In [52]:
apac_sales = df[df['Market'] == 'APAC'].groupby(['Year', 'Country'], as_index=False)['Sales'].sum()

apac_sales['Market Share'] = apac_sales.groupby('Year')['Sales'].transform(lambda x: (x / x.sum()) * 100)

top_5_countries = apac_sales[apac_sales['Year'] == 2011].nlargest(5, 'Sales')['Country'].tolist()

apac_top5_sales = apac_sales[apac_sales['Country'].isin(top_5_countries)]

fig = px.line(
    apac_top5_sales, x='Year', y='Market Share', color='Country', markers=True,
    labels={'Market Share': '시장 점유율 (%)', 'Year': '년도', 'Country': '국가'}
)

fig.update_layout(
    title=dict(text="APAC 시장의 매출 기준 시장 점유율 상위 5개 국가",
               x=0.5),
    title_font_size=20,
    plot_bgcolor='white'
)

fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='lightgray', tickfont=dict(color='gray'))
fig.update_xaxes(tickmode="linear", dtick=1, tickfont=dict(color='gray'))

fig.show()


### 호주, 중국의 연도별 매출, 이익

In [53]:
df['Order.Date'] = pd.to_datetime(df['Order.Date'])
df['Quarter'] = df['Order.Date'].dt.quarter
df['Month'] = df['Order.Date'].dt.month
df['Year'] = df['Order.Date'].dt.year

df_filtered = df[df["Country"].isin(['Australia', 'China'])]

sales_profit = df_filtered.groupby(["Year", "Country"]).agg({"Sales": "sum", "Profit": "sum"}).reset_index()

fig_sales = px.bar(
    sales_profit, x="Year", y="Sales", color="Country", barmode="group",
    labels={"Sales": "총 매출 ($)", "Year": "년도"},
    text=sales_profit["Sales"].apply(lambda x: f"${x:,.0f}")
)

fig_sales.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='lightgray') 
fig_sales.update_traces(textposition="outside")
fig_sales.update_layout(
    title=dict(text="호주 vs 중국 연간 매출 비교",
               x=0.5),
    title_font_size=20,
    plot_bgcolor="white",
    legend_title_text="국가"
)
fig_sales.show()

fig_profit = px.bar(
    sales_profit, x="Year", y="Profit", color="Country", barmode="group",
    labels={"Profit": "총 이익 ($)", "Year": "년도"},
    text=sales_profit["Profit"].apply(lambda x: f"${x:,.0f}")
)

fig_profit.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor="lightgray")  
fig_profit.update_traces(textposition="outside")
fig_profit.update_layout(
    title=dict(text="호주 vs 중국 연간 이익 비교",
               x=0.5),
    title_font_size=20,
    plot_bgcolor="white",
    legend_title_text="국가"
)
fig_profit.show()


### 호주, 중국 카테고리별 매출 비교

In [54]:
australia_china_df = df[df['Country'].isin(['Australia', 'China'])]

category_sales_comparison = australia_china_df.groupby(['Year', 'Country', 'Category'])['Sales'].sum().reset_index()

fig = px.bar(
    category_sales_comparison, x='Year', y='Sales', color='Category', barmode='stack',
    facet_col='Country', facet_col_spacing=0.08, 
    title="호주 vs 중국의 연간 카테고리별 매출",
    labels={'Sales': '총 매출 ($)', 'Year': '년도', 'Category': '카테고리', 'Country': '국가'},
    text=category_sales_comparison['Sales'].apply(lambda x: f"${x:,.0f}") 
)

fig.update_traces(textposition='inside', textangle=0)  # textangle=0 설정하여 숫자를 가로로 정렬

fig.update_layout(
    title_font_size=20,
    title_font_family="Liberation Serif",
    title_font_color="#5b5b5b",
    plot_bgcolor='white'
)

fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='lightgray', tickfont=dict(color='gray'))
fig.update_xaxes(tickmode="linear", dtick=1, tickfont=dict(color='gray'))

fig.show()


### 호주, 중국 연도별 평균 할인율 비교

In [55]:
df_filtered = df[df["Country"].isin(["Australia", "China"])]

discount_comparison = df_filtered.groupby(["Year", "Country"])["Discount"].mean().reset_index()

fig = px.bar(
    discount_comparison, x="Year", y="Discount", color="Country", barmode="group",
    labels={"Discount": "평균 할인율 (%)", "Year": "년도"},
    text=discount_comparison["Discount"].apply(lambda x: f"{x:.1%}")  
)

fig.update_traces(textposition="outside")

fig.update_layout(
    title=dict(text="호주 vs 중국 연간 평균 할인율 비교",
               x=0.5),
    title_font_size=20,
    plot_bgcolor="white",
    legend_title_text="국가",
    yaxis=dict(
        showgrid=True, gridwidth=0.5, gridcolor="lightgray"
    )

)

fig.show()


In [56]:
china_aus = df[df.Country.isin(['China', 'Australia'])]

aus = df[df.Country=='Australia']
aus.groupby('Year')[['Discount', 'Profit']].mean().corr()

,Discount,Profit
Discount,1.000000,-0.957376
Profit,-0.957376,1.000000


In [57]:
china = df[df.Country=='China']
china.groupby('Year')[['Discount', 'Profit']].mean().corr()

,Discount,Profit
Discount,1.00000,0.65255
Profit,0.65255,1.00000


In [58]:
import scipy.stats as stats
corr, p_value = stats.pearsonr(aus['Discount'], aus['Profit'])
print(p_value)

7.448342047214582e-57


In [59]:
import statsmodels.api as sm
X = aus['Discount']  
y = aus['Profit']  

X = sm.add_constant(X)  
model = sm.OLS(y, X).fit()  
print(model.summary())  

                            OLS Regression Results                            
Dep. Variable:                 Profit   R-squared:                       0.085
Model:                            OLS   Adj. R-squared:                  0.085
Method:                 Least Squares   F-statistic:                     264.1
Date:                Sat, 10 May 2025   Prob (F-statistic):           7.45e-57
Time:                        16:11:28   Log-Likelihood:                -18118.
No. Observations:                2837   AIC:                         3.624e+04
Df Residuals:                    2835   BIC:                         3.625e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         90.7715      4.287     21.175      0.0

### 국가, 카테고리별 할인율 그래프

In [60]:
df_copy = df[['Country', 'Category', 'Sales', 'Discount', 'Quantity', 'Profit']].copy()
df_copy = df_copy[df_copy['Country'].isin(['Australia', 'China'])]  

color_map = {"China": "#E74C3C", "Australia": "#3498DB"}

fig = px.scatter(df_copy, 
                 x="Discount", y="Profit", 
                 color="Country", 
                 size="Sales", 
                 facet_col="Category",  
                 title="국가, 카테고리별 할인율과 이익 비교",
                 color_discrete_map=color_map)  

fig.update_layout(width=1000, height=800, 
                  font=dict(size=12)) 

fig.show()



In [61]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.085
Model:                            OLS   Adj. R-squared:                  0.085
Method:                 Least Squares   F-statistic:                     264.1
Date:                Sat, 10 May 2025   Prob (F-statistic):           7.45e-57
Time:                        16:11:28   Log-Likelihood:                -18118.
No. Observations:                2837   AIC:                         3.624e+04
Df Residuals:                    2835   BIC:                         3.625e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         90.7715      4.287     21.175      0.000      82.366      99.177
Discount    -377.2381     23.212    -16.252      0.000    -422.752    -331.724
==============================================================================
Omnibus:                     2460.998   Durbin-Watson:                   1.935
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           228343.790
Skew:                           3.629   Prob(JB):                         0.00
Kurtosis:                      46.348   Cond. No.                         8.78
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### 최적 할인율 상한선 그래프

In [62]:
australia_df = df[df['Country'] == 'Australia']
australia_df['Discount_ratio'] = australia_df['Discount'] * 100

discount_profit_trend = australia_df.groupby(['Year', 'Quarter']).agg({'Discount_ratio': 'mean', 'Profit': 'sum', 'Sales': 'sum'}).reset_index()
discount_profit_trend['Profit Margin'] = discount_profit_trend['Profit'] / discount_profit_trend['Sales'] * 100
discount_profit_trend['Year-Quarter'] = discount_profit_trend['Year'].astype(str) + '-' +discount_profit_trend['Quarter'].astype(str) + 'Q'

avg_margin = discount_profit_trend['Profit Margin'].mean()


# 회귀선
X = discount_profit_trend['Discount_ratio']
y = discount_profit_trend['Profit Margin']
X_const = sm.add_constant(X)
model = sm.OLS(y, X_const).fit()

# 이익률과 회귀선이 만나는 지점 계산
slope = model.params[1]
intercept = model.params[0]
optimal_discount = (avg_margin - intercept) / slope

fig = px.scatter(
    discount_profit_trend, x='Discount_ratio', y='Profit Margin', text=discount_profit_trend['Year-Quarter'],
    title='호주의 분기 할인율과 이익률의 관계',
    labels={'Discount_ratio': '평균 할인율 (%)', 'Profit Margin': '이익률 (%)'},
    trendline='ols',  
    trendline_color_override='red'
)

fig.add_vline(
    x=optimal_discount,
    line_dash="dot",
    line_color="green",
    annotation_text=f"최소 목표 이익률 만족 할인율 ≈ {optimal_discount:.2f}%",
    annotation_position="top right"
)

fig.update_traces(
    marker=dict(size=10), textposition='top center',
    textfont=dict(size=12),
    cliponaxis=False) 

fig.show()

C:\Users\xodbs\AppData\Local\Temp\ipykernel_14284\2781203004.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\xodbs\AppData\Local\Temp\ipykernel_14284\2781203004.py:18: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\xodbs\AppData\Local\Temp\ipykernel_14284\2781203004.py:19: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [63]:
# 평균 이익률 기준선
baseline_margin = 11.23

# 조건을 만족하는 연도들
valid_years = discount_profit_trend[discount_profit_trend['Profit Margin'] >= baseline_margin]

# 하한 할인율
lower_discount = valid_years['Discount_ratio'].min()

lower_discount

9.91150442477876

### 최적 할인율 그래프

In [64]:
X = discount_profit_trend['Discount_ratio']
y = discount_profit_trend['Profit Margin']
X_const = sm.add_constant(X)
model = sm.OLS(y, X_const).fit()

fig = px.scatter(
    discount_profit_trend, 
    x='Discount_ratio', 
    y='Profit Margin', 
    text=discount_profit_trend['Year'],
    labels={'Discount_ratio': '할인율 (%)', 'Profit Margin': '이익률 (%)'},
    trendline='ols',
    trendline_color_override="red",
    color_discrete_sequence=["#636EFA"]
)

fig.add_vrect(
    x0=lower_discount, x1=14.08,
    fillcolor="green", opacity=0.2,
    layer="below", line_width=0,
    annotation_text="최적 할인율 범위", annotation_position="top left"
)

fig.update_layout(
    title=dict(text="호주의 할인율과 이익률 관계 및 최적 할인율 구간",
               x=0.5)
)
fig.update_traces(marker=dict(size=10), textposition='top right')
fig.show()


In [65]:
australia_china_df = df[df['Country'].isin(['Australia', 'China']) ]

discount_profit_trend = australia_china_df.groupby(['Country', 'Year']).agg({
    'Discount': 'mean',
    'Profit': 'sum',
    'Sales': 'sum'
}).reset_index()

discount_profit_trend['Profit Margin'] = discount_profit_trend['Profit'] / discount_profit_trend['Sales'] *100
discount_profit_trend['Profit Margin Text'] = discount_profit_trend['Profit Margin'].apply(lambda x: f"{x:.1f}%")

# 그래프 그리기
fig = px.line(
    discount_profit_trend,
    x='Year',
    y='Profit Margin',
    color='Country',
    markers=True,
    labels={'Profit Margin': 'Profit Margin (%)'},
    text='Profit Margin Text',  # ← 여기 수정됨
    color_discrete_sequence=["#636EFA", "#EF553B"]
)

fig.update_traces(marker=dict(size=10), textposition='top right')
fig.update_layout(
    plot_bgcolor='white',
    title=dict(text='국가별 연도별 이익률 추이', x=0.5),
    yaxis_title='Profit Margin (%)'
)

fig.show()

In [66]:
australia_china_df

,Category,City,Country,Customer.ID,Customer.Name,Discount,Market,记录数,Order.Date,Order.ID,...,Ship.Date,Ship.Mode,Shipping.Cost,State,Sub.Category,Year,Market2,weeknum,Quarter,Month
3329,Office Supplies,Jieyang,China,PS-190451,Penelope Sewall,0.0,APAC,1,2011-04-16,IN-2011-74490,...,2011-04-17 00:00:00.000,First Class,4.20,Guangdong,Binders,2011,APAC,16,2,4
3330,Office Supplies,Jieyang,China,PS-190451,Penelope Sewall,0.0,APAC,1,2011-04-16,IN-2011-74490,...,2011-04-17 00:00:00.000,First Class,10.67,Guangdong,Envelopes,2011,APAC,16,2,4
3331,Office Supplies,Jieyang,China,PS-190451,Penelope Sewall,0.0,APAC,1,2011-04-16,IN-2011-74490,...,2011-04-17 00:00:00.000,First Class,36.53,Guangdong,Paper,2011,APAC,16,2,4
3336,Office Supplies,Taizhou,China,EM-141401,Eugene Moren,0.0,APAC,1,2011-06-03,IN-2011-50585,...,2011-06-05 00:00:00.000,Second Class,7.85,Jiangsu,Storage,2011,APAC,23,2,6
3342,Office Supplies,Shenyang,China,CD-127901,Cynthia Delaney,0.0,APAC,1,2011-07-20,IN-2011-44369,...,2011-07-23 00:00:00.000,First Class,8.95,Liaoning,Binders,2011,APAC,30,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46089,Furniture,Sydney,Australia,AZ-107501,Annie Zypern,0.1,APAC,1,2014-10-14,IN-2014-69604,...,2014-10-20 00:00:00.000,Standard Class,24.45,New South Wales,Chairs,2014,APAC,42,4,10
46090,Furniture,Sydney,Australia,BE-113351,Bill Eplett,0.1,APAC,1,2014-10-25,IN-2014-41709,...,2014-10-29 00:00:00.000,Standard Class,93.30,New South Wales,Bookcases,2014,APAC,43,4,10
46091,Office Supplies,Sydney,Australia,BE-113351,Bill Eplett,0.1,APAC,1,2014-10-25,IN-2014-41709,...,2014-10-29 00:00:00.000,Standard Class,2.34,New South Wales,Art,2014,APAC,43,4,10
46092,Furniture,Sydney,Australia,IG-150851,Ivan Gibson,0.1,APAC,1,2014-10-28,IN-2014-50011,...,2014-11-04 00:00:00.000,Standard Class,61.46,New South Wales,Chairs,2014,APAC,44,4,10
